In [1]:
!pip install -q pyiceberg

In [2]:
from pyiceberg.catalog.rest import RestCatalog
import logging

# logging.basicConfig(level=logging.DEBUG)

import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

CATALOG_URL = "http://lakekeeper:8181/catalog"
DEMO_WAREHOUSE = "irisa-ot"

catalog = RestCatalog(
    name="banking_catalog",
    warehouse=DEMO_WAREHOUSE,
    uri=CATALOG_URL,
    token="dummy",
)


In [3]:
# Create a new namespace if it doesn't already exist
test_namespace = ("pyiceberg_namespace",)
if test_namespace not in catalog.list_namespaces():
    catalog.create_namespace(test_namespace)


In [4]:
# Write data
test_table = ("pyiceberg_namespace", "my_table")
df = pd.DataFrame(
    {
        "id": [1, 2, 3],
        "data": ["a", "b", "c"],
    }
)
pa_df = pa.Table.from_pandas(df)

if test_table in catalog.list_tables(namespace=test_namespace):
    catalog.drop_table(test_table)

table = catalog.create_table(
    test_table,
    schema=pa_df.schema,
)

table.append(pa_df)

In [5]:
# Read data
table = catalog.load_table(test_table)
table.scan().to_pandas()

,id,data
0,1,a
1,2,b
2,3,c
